In [18]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)

In [19]:
import pandas as pd
from common.extractor.alpaca_extractor import AlpacaExtractor
from common.processor.processor import Processor

In [20]:
alpaca = AlpacaExtractor(paper=False)

In [21]:
orders = alpaca.orders()

In [22]:
orders = pd.DataFrame(orders)

In [23]:
orders["asset_class"].unique()

array(['us_equity', 'crypto'], dtype=object)

In [24]:
trades = orders[orders["asset_class"]=="us_equity"][["filled_at","symbol","notional","filled_qty","filled_avg_price","side"]]
trades["date"] = [x.split("T")[0] for x in trades["filled_at"]]
trades = Processor.utc_date(trades)

trades["filled_avg_price"] = trades["filled_avg_price"].astype(float)
trades["filled_qty"] = trades["filled_qty"].astype(float)
stuff = []
for symbol in trades["symbol"].unique():
    asset_trades = trades[trades["symbol"]==symbol].sort_values("date")
    asset_trades["sell_date"] = asset_trades["filled_at"].shift(-1)
    asset_trades["sell_avg_price"] = asset_trades["filled_avg_price"].shift(-1)
    asset_trades = asset_trades[asset_trades["side"]=="buy"].dropna().drop("filled_at",axis=1)
    asset_trades["pnl"] = asset_trades["sell_avg_price"] / asset_trades["filled_avg_price"]
    stuff.append(asset_trades)

In [25]:
analysis = pd.concat(stuff)
analysis.sort_values("date",ascending=False).head(20)

,symbol,notional,filled_qty,filled_avg_price,side,date,sell_date,sell_avg_price,pnl
273,OI,5,0.373469,13.3880,buy,2025-05-05 00:00:00+00:00,2025-05-05T18:08:52.21335Z,13.3720,0.998805
271,MRNA,542.75,15.131029,35.8700,buy,2025-01-21 00:00:00+00:00,2025-01-24T19:05:46.707Z,41.4134,1.154541
269,MRNA,450,12.791359,35.1800,buy,2025-01-15 00:00:00+00:00,2025-01-17T19:06:03.712Z,34.0824,0.968800
268,MRNA,106.07,3.134054,33.8432,buy,2025-01-13 00:00:00+00:00,2025-01-15T14:30:01.11Z,35.1800,1.039500
266,CE,107.3,1.621924,66.1560,buy,2025-01-06 00:00:00+00:00,2025-01-10T19:05:57.587Z,65.4067,0.988674
264,ES,27.41,0.478393,57.2960,buy,2024-12-30 00:00:00+00:00,2025-01-06T14:30:01.103581Z,57.1000,0.996579
125,NFE,11.86,1.209464,9.8060,buy,2024-09-23 00:00:00+00:00,2024-09-25T19:49:59.716Z,9.5064,0.969447
130,SMCI,11.86,0.025613,463.0380,buy,2024-09-23 00:00:00+00:00,2024-09-25T19:49:59.731512Z,459.5200,0.992402
121,SIRI,11.86,0.463499,25.5880,buy,2024-09-23 00:00:00+00:00,2024-09-25T19:49:59.729644Z,24.0520,0.939972
122,WOLF,11.86,1.486588,7.9780,buy,2024-09-23 00:00:00+00:00,2024-09-25T19:49:59.715Z,7.9122,0.991752


In [26]:
portfolio = analysis.groupby(["date"]).agg({"pnl":"mean"}).reset_index()
portfolio["pv"] = portfolio["pnl"].cumprod()